In [1501]:
import numpy as np
import geopandas
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from itertools import chain
import io
%matplotlib inline

In [1547]:
# To load comma-separated values data into pandas 
loc_csv = r'X:\Groups\N41\GROUP-ORGA\MOBILITY_cluster\Activities\Traffic_model\a_matsim_pop_gen\matsim_pop_gen_Strasbourg\output\LOA\location_of_activity.csv'
df = pd.read_csv(loc_csv, sep = ',')
df

,index,sector,sector_fine,sample_id,person_id,sex,age,enrollment,license,occupation,...,iris_chain,primary_activity,Home,Work,Accompany,Shopping,Leisure,School,Other,University
0,0,1,4,18,1,M,30_44,secondary_cap,yes,full_time,...,671300102>674822204>671300102,Work,671300102,674822204,NaN,NaN,NaN,NaN,NaN,NaN
1,1,1,4,18,2,F,30_44,secondary_cap,yes,full_time,...,671300102>671300102>673360000>671300102>671300...,Work,671300102,673360000,671300102,671300103,671300102,NaN,NaN,NaN
2,2,1,4,18,3,M,15_29,secondary_bep,yes,full_time,...,671300102>671300102>671300101>671300102>671300...,Work,671300102,"671300102, 671300101",NaN,NaN,671300104,NaN,NaN,NaN
3,3,1,4,18,4,M,00_14,in_school,no,pupil,...,671300102>671300102>671300102>671300102>671300102,School,671300102,NaN,NaN,NaN,671300102,671300102,NaN,NaN
4,4,1,4,59,1,F,15_29,secondary_cap,no,at_home,...,671300102>671300103>671300102,Shopping,671300102,NaN,NaN,671300103,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9857,10301,68,47,128,1,M,45_59,bac+2,yes,full_time,...,675470000>674620105>674620109>675470000,Work,675470000,674620105,NaN,674620109,NaN,NaN,NaN,NaN
9858,10302,68,47,128,2,F,45_59,secondary_bep,yes,full_time,...,675470000>670730000>675470000>674620109>674620...,Work,675470000,670730000,674620102,674620109,NaN,NaN,NaN,NaN
9859,10303,68,47,128,3,M,15_29,in_school,no,pupil,...,675470000>674620102>675470000>675470000>675470000,School,675470000,NaN,NaN,NaN,675470000,674620102,NaN,NaN
9860,10304,68,47,137,2,F,60_74,primary,yes,retired,...,675470000>673110000>674620106>674620106>674620...,Shopping,675470000,NaN,673110000,674620106,674620106,NaN,NaN,NaN


In [1503]:
columns =['index','sample_id', 'person_id', 'activity_chain','iris_chain','primary_activity', 'Work','School', 'University','Shopping', 'Leisure', 'Accompany', 'Other', 'Home']
df = pd.read_csv(loc_csv, sep = ',',skipinitialspace=True,  dtype=str, usecols=columns)
# Add index name
#df = pd.DataFrame(df).rename_axis('idx_org', axis=1)
df.head()

,index,sample_id,person_id,activity_chain,iris_chain,primary_activity,Home,Work,Accompany,Shopping,Leisure,School,Other,University
0,0,018,01,Home>Work>Home,671300102>674822204>671300102,Work,671300102,674822204,NaN,NaN,NaN,NaN,NaN,NaN
1,1,018,02,Home>Accompany>Work>Accompany>Shopping>Accompa...,671300102>671300102>673360000>671300102>671300...,Work,671300102,673360000,671300102,671300103,671300102,NaN,NaN,NaN
2,2,018,03,Home>Work>Work>Home>Work>Work>Work>Home>Leisur...,671300102>671300102>671300101>671300102>671300...,Work,671300102,"671300102, 671300101",NaN,NaN,671300104,NaN,NaN,NaN
3,3,018,04,Home>School>Leisure>School>Home,671300102>671300102>671300102>671300102>671300102,School,671300102,NaN,NaN,NaN,671300102,671300102,NaN,NaN
4,4,059,01,Home>Shopping>Home,671300102>671300103>671300102,Shopping,671300102,NaN,NaN,671300103,NaN,NaN,NaN,NaN


### Split activity_chain and iris_chain into rows

In [1504]:
dfs = df.set_index(['index','sample_id', 'person_id','primary_activity', 
                    'Work','School', 'University','Shopping', 'Leisure', 'Accompany', 'Other', 'Home']).apply(lambda x: x.str.split('>').explode()).reset_index()
#dfs.index.name = 'index'
dfs.head()

,index,sample_id,person_id,primary_activity,Work,School,University,Shopping,Leisure,Accompany,Other,Home,activity_chain,iris_chain
0,0,018,01,Work,674822204,NaN,NaN,NaN,NaN,NaN,NaN,671300102,Home,671300102
1,0,018,01,Work,674822204,NaN,NaN,NaN,NaN,NaN,NaN,671300102,Work,674822204
2,0,018,01,Work,674822204,NaN,NaN,NaN,NaN,NaN,NaN,671300102,Home,671300102
3,1,018,02,Work,673360000,NaN,NaN,671300103,671300102,671300102,NaN,671300102,Home,671300102
4,1,018,02,Work,673360000,NaN,NaN,671300103,671300102,671300102,NaN,671300102,Accompany,671300102


### 1. Find the duplicated  HOME activities in the activity chain

### 1.1. Remove last activity home in simple approach 

In [1505]:
df['iris_chain_s']=df.iris_chain.str.split(">").str[:-1]
df['iris_chain_s']

0                                  [671300102, 674822204]
1       [671300102, 671300102, 673360000, 671300102, 6...
2       [671300102, 671300102, 671300101, 671300102, 6...
3            [671300102, 671300102, 671300102, 671300102]
4                                  [671300102, 671300103]
                              ...                        
9857                    [675470000, 674620105, 674620109]
9858    [675470000, 670730000, 675470000, 674620109, 6...
9859         [675470000, 674620102, 675470000, 675470000]
9860    [675470000, 673110000, 674620106, 674620106, 6...
9861    [675470000, 675470000, 674860000, 675470000, 6...
Name: iris_chain_s, Length: 9862, dtype: object

### 1.2. Convert a list to df and assign column name

In [1506]:
dfn = pd.DataFrame(df).rename_axis('idx', axis=1)
# Add index to have the orginal id
#dfn.index.name = 'idx'
dfn.head()

idx,index,sample_id,person_id,activity_chain,iris_chain,primary_activity,Home,Work,Accompany,Shopping,Leisure,School,Other,University,iris_chain_s
0,0,018,01,Home>Work>Home,671300102>674822204>671300102,Work,671300102,674822204,NaN,NaN,NaN,NaN,NaN,NaN,"[671300102, 674822204]"
1,1,018,02,Home>Accompany>Work>Accompany>Shopping>Accompa...,671300102>671300102>673360000>671300102>671300...,Work,671300102,673360000,671300102,671300103,671300102,NaN,NaN,NaN,"[671300102, 671300102, 673360000, 671300102, 6..."
2,2,018,03,Home>Work>Work>Home>Work>Work>Work>Home>Leisur...,671300102>671300102>671300101>671300102>671300...,Work,671300102,"671300102, 671300101",NaN,NaN,671300104,NaN,NaN,NaN,"[671300102, 671300102, 671300101, 671300102, 6..."
3,3,018,04,Home>School>Leisure>School>Home,671300102>671300102>671300102>671300102>671300102,School,671300102,NaN,NaN,NaN,671300102,671300102,NaN,NaN,"[671300102, 671300102, 671300102, 671300102]"
4,4,059,01,Home>Shopping>Home,671300102>671300103>671300102,Shopping,671300102,NaN,NaN,671300103,NaN,NaN,NaN,NaN,"[671300102, 671300103]"


In [1507]:
dfs.reset_index(level=0, inplace=False)
dfs.head(3)

,index,sample_id,person_id,primary_activity,Work,School,University,Shopping,Leisure,Accompany,Other,Home,activity_chain,iris_chain
0,0,018,01,Work,674822204,NaN,NaN,NaN,NaN,NaN,NaN,671300102,Home,671300102
1,0,018,01,Work,674822204,NaN,NaN,NaN,NaN,NaN,NaN,671300102,Work,674822204
2,0,018,01,Work,674822204,NaN,NaN,NaN,NaN,NaN,NaN,671300102,Home,671300102


## 2 Remove duplicated home with checking 

In [1508]:
columns =['index','sample_id', 'person_id', 'activity_chain','iris_chain','primary_activity']
df = pd.read_csv(loc_csv, sep = ',',skipinitialspace=True,  dtype=str, usecols=columns)
# Add index name
#df = pd.DataFrame(df).rename_axis('idx_org', axis=1)
df.head()

,index,sample_id,person_id,activity_chain,iris_chain,primary_activity
0,0,018,01,Home>Work>Home,671300102>674822204>671300102,Work
1,1,018,02,Home>Accompany>Work>Accompany>Shopping>Accompa...,671300102>671300102>673360000>671300102>671300...,Work
2,2,018,03,Home>Work>Work>Home>Work>Work>Work>Home>Leisur...,671300102>671300102>671300101>671300102>671300...,Work
3,3,018,04,Home>School>Leisure>School>Home,671300102>671300102>671300102>671300102>671300102,School
4,4,059,01,Home>Shopping>Home,671300102>671300103>671300102,Shopping


### Convert df['iris_chain'] to a  list 

In [1509]:
df['iris_chain_n']= df['iris_chain'].apply(lambda x: x.replace('>',',')) 
df.head(3)

,index,sample_id,person_id,activity_chain,iris_chain,primary_activity,iris_chain_n
0,0,018,01,Home>Work>Home,671300102>674822204>671300102,Work,"671300102,674822204,671300102"
1,1,018,02,Home>Accompany>Work>Accompany>Shopping>Accompa...,671300102>671300102>673360000>671300102>671300...,Work,"671300102,671300102,673360000,671300102,671300..."
2,2,018,03,Home>Work>Work>Home>Work>Work>Work>Home>Leisur...,671300102>671300102>671300101>671300102>671300...,Work,"671300102,671300102,671300101,671300102,671300..."


In [1510]:
df['iris_chain_n'] = df['iris_chain_n'].apply(lambda x: x.split(','))
df.head(3)

,index,sample_id,person_id,activity_chain,iris_chain,primary_activity,iris_chain_n
0,0,018,01,Home>Work>Home,671300102>674822204>671300102,Work,"[671300102, 674822204, 671300102]"
1,1,018,02,Home>Accompany>Work>Accompany>Shopping>Accompa...,671300102>671300102>673360000>671300102>671300...,Work,"[671300102, 671300102, 673360000, 671300102, 6..."
2,2,018,03,Home>Work>Work>Home>Work>Work>Work>Home>Leisur...,671300102>671300102>671300101>671300102>671300...,Work,"[671300102, 671300102, 671300101, 671300102, 6..."


In [1511]:
df['activity_chain_n']= df['activity_chain'].apply(lambda x: x.replace('>',',')) 
df.head(3)

,index,sample_id,person_id,activity_chain,iris_chain,primary_activity,iris_chain_n,activity_chain_n
0,0,018,01,Home>Work>Home,671300102>674822204>671300102,Work,"[671300102, 674822204, 671300102]","Home,Work,Home"
1,1,018,02,Home>Accompany>Work>Accompany>Shopping>Accompa...,671300102>671300102>673360000>671300102>671300...,Work,"[671300102, 671300102, 673360000, 671300102, 6...","Home,Accompany,Work,Accompany,Shopping,Accompa..."
2,2,018,03,Home>Work>Work>Home>Work>Work>Work>Home>Leisur...,671300102>671300102>671300101>671300102>671300...,Work,"[671300102, 671300102, 671300101, 671300102, 6...","Home,Work,Work,Home,Work,Work,Work,Home,Leisur..."


In [1512]:
df['activity_chain_n'] = df['activity_chain_n'].apply(lambda x: x.split(','))
df.head(3)

,index,sample_id,person_id,activity_chain,iris_chain,primary_activity,iris_chain_n,activity_chain_n
0,0,018,01,Home>Work>Home,671300102>674822204>671300102,Work,"[671300102, 674822204, 671300102]","[Home, Work, Home]"
1,1,018,02,Home>Accompany>Work>Accompany>Shopping>Accompa...,671300102>671300102>673360000>671300102>671300...,Work,"[671300102, 671300102, 673360000, 671300102, 6...","[Home, Accompany, Work, Accompany, Shopping, A..."
2,2,018,03,Home>Work>Work>Home>Work>Work>Work>Home>Leisur...,671300102>671300102>671300101>671300102>671300...,Work,"[671300102, 671300102, 671300101, 671300102, 6...","[Home, Work, Work, Home, Work, Work, Work, Hom..."


### Remove duplicated value at the last position from the list in each row: lambda 

In [1513]:
col ='iris_chain_n'
df[col] = df.apply(lambda row: row[col][:-1] if row[col][0] ==row[col][-1]
                              else row[col],  axis=1)
df.head(2)

,index,sample_id,person_id,activity_chain,iris_chain,primary_activity,iris_chain_n,activity_chain_n
0,0,018,01,Home>Work>Home,671300102>674822204>671300102,Work,"[671300102, 674822204]","[Home, Work, Home]"
1,1,018,02,Home>Accompany>Work>Accompany>Shopping>Accompa...,671300102>671300102>673360000>671300102>671300...,Work,"[671300102, 671300102, 673360000, 671300102, 6...","[Home, Accompany, Work, Accompany, Shopping, A..."


In [1514]:
col ='activity_chain_n'
df[col] = df.apply(lambda row: row[col][:-1] if row[col][0] ==row[col][-1]
                              else row[col],  axis=1)
df.head(2)

,index,sample_id,person_id,activity_chain,iris_chain,primary_activity,iris_chain_n,activity_chain_n
0,0,018,01,Home>Work>Home,671300102>674822204>671300102,Work,"[671300102, 674822204]","[Home, Work]"
1,1,018,02,Home>Accompany>Work>Accompany>Shopping>Accompa...,671300102>671300102>673360000>671300102>671300...,Work,"[671300102, 671300102, 673360000, 671300102, 6...","[Home, Accompany, Work, Accompany, Shopping, A..."


### Save current df to csv

In [1515]:
path_inter= r'..\output\_results\loc_inetrmidiate.csv'
df.to_csv(path_inter, index=False)

In [1541]:
dfn = pd.read_csv(path_inter, sep = ',',skipinitialspace=True,  dtype=str, index_col=['index'])
dfn.reset_index(level=0, inplace=True)
# Add index name
#dfn = pd.DataFrame(df).rename_axis('idx_org', axis=1)
dfn.head(3)

,index,sample_id,person_id,activity_chain,iris_chain,primary_activity,iris_chain_n,activity_chain_n
0,0,018,01,Home>Work>Home,671300102>674822204>671300102,Work,"['671300102', '674822204']","['Home', 'Work']"
1,1,018,02,Home>Accompany>Work>Accompany>Shopping>Accompa...,671300102>671300102>673360000>671300102>671300...,Work,"['671300102', '671300102', '673360000', '67130...","['Home', 'Accompany', 'Work', 'Accompany', 'Sh..."
2,2,018,03,Home>Work>Work>Home>Work>Work>Work>Home>Leisur...,671300102>671300102>671300101>671300102>671300...,Work,"['671300102', '671300102', '671300101', '67130...","['Home', 'Work', 'Work', 'Home', 'Work', 'Work..."


### Remove [] from list

In [1542]:
dfn['activity_chain_n'] = dfn['activity_chain_n'].apply(lambda x:  str(x).strip('[]'))
dfn['iris_chain_n'] = dfn['iris_chain_n'].apply(lambda x:  str(x).strip('[]'))
#df['activity_chain_n'] =df['activity_chain_n'].apply(lambda x: x[1:-1].split(','))
#dfn['activity_chain_n'] = dfn['activity_chain_n'].astype(str).str[1:-1]
#dfn['iris_chain_n'] = dfn['iris_chain_n'].astype(str).str[1:-1]
#df['activity_chain_n']= df['activity_chain_n'].apply(lambda x: x.replace('[','').replace(']','')) 
#df['iris_chain_n']= df['iris_chain_n'].apply(lambda x: x.replace('[','').replace(']','')) 
dfn.head(3)

,index,sample_id,person_id,activity_chain,iris_chain,primary_activity,iris_chain_n,activity_chain_n
0,0,018,01,Home>Work>Home,671300102>674822204>671300102,Work,"'671300102', '674822204'","'Home', 'Work'"
1,1,018,02,Home>Accompany>Work>Accompany>Shopping>Accompa...,671300102>671300102>673360000>671300102>671300...,Work,"'671300102', '671300102', '673360000', '671300...","'Home', 'Accompany', 'Work', 'Accompany', 'Sho..."
2,2,018,03,Home>Work>Work>Home>Work>Work>Work>Home>Leisur...,671300102>671300102>671300101>671300102>671300...,Work,"'671300102', '671300102', '671300101', '671300...","'Home', 'Work', 'Work', 'Home', 'Work', 'Work'..."


### Put again '>' into the list

In [1543]:
dfn['activity_chain_n']= dfn['activity_chain_n'].apply(lambda row: row.replace(",", ">").replace("''", ""))
dfn['iris_chain_n']= dfn['iris_chain_n'].apply(lambda row: row.replace(",", ">"))
df.head(3)

,index,sample_id,person_id,activity_chain,iris_chain,primary_activity,iris_chain_n,activity_chain_n
0,0,018,01,Home>Work>Home,671300102>674822204>671300102,Work,'671300102'> '674822204','Home'> 'Work'
1,1,018,02,Home>Accompany>Work>Accompany>Shopping>Accompa...,671300102>671300102>673360000>671300102>671300...,Work,'671300102'> '671300102'> '673360000'> '671300...,'Home'> 'Accompany'> 'Work'> 'Accompany'> 'Sho...
2,2,018,03,Home>Work>Work>Home>Work>Work>Work>Home>Leisur...,671300102>671300102>671300101>671300102>671300...,Work,'671300102'> '671300102'> '671300101'> '671300...,'Home'> 'Work'> 'Work'> 'Home'> 'Work'> 'Work'...


In [1544]:
dfn.columns

Index(['index', 'sample_id', 'person_id', 'activity_chain', 'iris_chain',
       'primary_activity', 'iris_chain_n', 'activity_chain_n'],
      dtype='object')

In [1549]:
dfs = df.set_index(['index', 'sample_id', 'person_id', 'activity_chain', 'iris_chain',
                   'primary_activity','iris_chain_n','activity_chain_n'])
#dfs.index.name = 'index'
dfs.head()

KeyError: "None of ['iris_chain_n', 'activity_chain_n'] are in the columns"

In [ ]:
dfs = df.set_index(['index', 'sample_id', 'person_id', 'activity_chain', 'iris_chain',
                   'primary_activity','activity_chain_n']).apply(lambda x: x.str.split('>').explode()).reset_index()
#dfs.index.name = 'index'
dfs.head()

### Remove duplicated value at teh last position from the list in each row: iteration 

In [1246]:
# Remove 
a_list=[]
df_test=dfn
for index, rows in df_test.iterrows():
    a= index, rows.iris_chain
    b= rows.iris_chain[0], rows.iris_chain[-1]
    if rows.iris_chain[0] ==rows.iris_chain[-1]:
        b= [index, rows.iris_chain[:-1]]
    else:
        b= [index, rows.iris_chain]
    a_list.append(b)
#a_list

### 2.4 Removing brackets from list inside pandas cell

In [1247]:
df_a = pd.DataFrame(a_list, columns= ['idx', 'activity'])
# remove list bracets from cells
df_a['activity'] = df_a['activity'].astype(str).str[1:-1]
#df_a['activity']= df_a['activity'].apply(lambda x: x.replace('[','').replace(']','')) 
df_a.head(3)

,idx,activity
0,0,71300102>674822204>67130010
1,1,71300102>671300102>673360000>671300102>6713001...
2,2,71300102>671300102>671300101>671300102>6713001...


### 3. Return list from series of comma-separated strings
### 3.1 Split 'iris_chain'  and activity_chain' into single rows (short version)

##### Shows columns names

In [1249]:
columnsl =list(df.columns) 
columnsl 

['index',
 'sample_id',
 'person_id',
 'activity_chain',
 'iris_chain',
 'primary_activity',
 'iris_chain_n',
 'activity_chain_n']

In [1250]:
print(df.index.name)

None


##### Convert index to a column, in the next step the index will be reseted

In [1251]:
df.reset_index(level=0, inplace=False)
df.head(3)

,index,sample_id,person_id,activity_chain,iris_chain,primary_activity,iris_chain_n,activity_chain_n
0,0,018,01,Home>Work>Home,671300102>674822204>671300102,Work,"[671300102, 674822204]","[Home, Work]"
1,1,018,02,Home>Accompany>Work>Accompany>Shopping>Accompa...,671300102>671300102>673360000>671300102>671300...,Work,"[671300102, 671300102, 673360000, 671300102, 6...","[Home, Accompany, Work, Accompany, Shopping, A..."
2,2,018,03,Home>Work>Work>Home>Work>Work>Work>Home>Leisur...,671300102>671300102>671300101>671300102>671300...,Work,"[671300102, 671300102, 671300101, 671300102, 6...","[Home, Work, Work, Home, Work, Work, Work, Hom..."


In [1252]:
df.columns.is_unique

True

In [1253]:
df.columns.value_counts()

index               1
iris_chain          1
sample_id           1
iris_chain_n        1
primary_activity    1
person_id           1
activity_chain      1
activity_chain_n    1
dtype: int64

In [1254]:
#dfs = df.assign(iris_chain=df['activity'].str.split(',')).explode('activity')
#dfs.index.name = 'index'
#dfs

In [1255]:
dfs = df.set_index(['index',
 'sample_id',
 'person_id',
 'activity_chain',
 'iris_chain',
 'primary_activity',]).apply(lambda x: x.str.split(',').explode()).reset_index()
#dfs.index.name = 'index'
dfs.head()

,index,sample_id,person_id,activity_chain,iris_chain,primary_activity,iris_chain_n,activity_chain_n
0,0,018,01,Home>Work>Home,671300102>674822204>671300102,Work,NaN,NaN
1,1,018,02,Home>Accompany>Work>Accompany>Shopping>Accompa...,671300102>671300102>673360000>671300102>671300...,Work,NaN,NaN
2,2,018,03,Home>Work>Work>Home>Work>Work>Work>Home>Leisur...,671300102>671300102>671300101>671300102>671300...,Work,NaN,NaN
3,3,018,04,Home>School>Leisure>School>Home,671300102>671300102>671300102>671300102>671300102,School,NaN,NaN
4,4,059,01,Home>Shopping>Home,671300102>671300103>671300102,Shopping,NaN,NaN


In [825]:
# To csv
dfs.to_csv(r'..\output\_results\loc_split_acticity_iris.csv')

In [826]:
# Save to xls
writer = pd.ExcelWriter(r'..\output\_results\loc_split_acticity_iris.xlsx')
dfs.to_excel(writer, '{}'.format('loc'))
#writer.save()
print('done')

done


### 2.2 Pivot of iris and activity_chain 

In [442]:
#create pivot table
piv_t = pd.pivot_table(data= dfs, 
                          index = ['iris_chain', 'activity_chain'], 
                          values =['org_id'],
                          #columns =['Work','School'],
                          aggfunc=['count'])
print(piv_t)

                           count
                          org_id
iris_chain activity_chain       
670010000  Accompany          14
           Home              127
           Leisure            18
           Other               1
           School             24
...                          ...
675580000  Home                4
           Leisure             3
           School              1
           Work                2
675600000  Work                1

[3289 rows x 1 columns]


In [443]:
# Save to xls
writer = pd.ExcelWriter(r'..\output\_results\pivot_iris_activity.xlsx')
piv_t.to_excel(writer, '{}'.format('loc'))
#writer.save()
print('done')

done


### 2.3 Pivot of activity_chain and iris

In [444]:
#create pivot table
piv_ai = pd.pivot_table(data= dfs, 
                          index = ['activity_chain','iris_chain'], 
                          values =['org_id'],
                          #columns =['Work','School'],
                          aggfunc=['count'])
print(piv_ai)

                           count
                          org_id
activity_chain iris_chain       
Accompany      670010000      14
               670020000       1
               670030000       7
               670100000       5
               670110000       6
...                          ...
Work           675510000      18
               675530000       1
               675540000       2
               675580000       2
               675600000       1

[3289 rows x 1 columns]


In [445]:
# Save to xls
writer = pd.ExcelWriter(r'..\output\_results\pivot_activity_iris.xlsx')
piv_ai.to_excel(writer, '{}'.format('loc'))
#writer.save()
print('done')

done


### 2.4 Split 'iris_chain' into single rows (incl. acivity columns)

In [175]:
columns =['sample_id', 'person_id', 'iris_chain','primary_activity', 'Work','School', 'University','Shopping', 'Leisure', 'Accompany', 'Other', 'Home']
df = pd.read_csv(loc_csv, sep = ',', skipinitialspace=True, usecols=columns)
df['person_id']=pd.to_numeric(df['person_id'])
df['sample_id']=pd.to_numeric(df['sample_id'])
# Add index to have the orginal id
df.head()
# Add index to have the orginal id
df.index.name = 'org_id'
df.head()

,sample_id,person_id,iris_chain,primary_activity,Home,Work,Accompany,Shopping,Leisure,School,Other,University
org_id,,,,,,,,,,,,
0,18,1,671300102>674822204>671300102,Work,671300102,674822204,NaN,NaN,NaN,NaN,NaN,NaN
1,18,2,671300102>671300102>673360000>671300102>671300...,Work,671300102,673360000,671300102,671300103,671300102,NaN,NaN,NaN
2,18,3,671300102>671300102>671300101>671300102>671300...,Work,671300102,"671300102, 671300101",NaN,NaN,671300104,NaN,NaN,NaN
3,18,4,671300102>671300102>671300102>671300102>671300102,School,671300102,NaN,NaN,NaN,671300102,671300102,NaN,NaN
4,59,1,671300102>671300103>671300102,Shopping,671300102,NaN,NaN,671300103,NaN,NaN,NaN,NaN


In [176]:
# Convert index to a column, in the next step the index will be reseted
df.reset_index(level=0, inplace=True) 
df.head()

,org_id,sample_id,person_id,iris_chain,primary_activity,Home,Work,Accompany,Shopping,Leisure,School,Other,University
0,0,18,1,671300102>674822204>671300102,Work,671300102,674822204,NaN,NaN,NaN,NaN,NaN,NaN
1,1,18,2,671300102>671300102>673360000>671300102>671300...,Work,671300102,673360000,671300102,671300103,671300102,NaN,NaN,NaN
2,2,18,3,671300102>671300102>671300101>671300102>671300...,Work,671300102,"671300102, 671300101",NaN,NaN,671300104,NaN,NaN,NaN
3,3,18,4,671300102>671300102>671300102>671300102>671300102,School,671300102,NaN,NaN,NaN,671300102,671300102,NaN,NaN
4,4,59,1,671300102>671300103>671300102,Shopping,671300102,NaN,NaN,671300103,NaN,NaN,NaN,NaN


In [177]:
dfs = df.set_index(['org_id','sample_id', 'person_id','primary_activity',
                    'Work','School', 'University','Shopping',
                    'Leisure', 'Accompany', 'Other', 'Home']).apply(lambda x: x.str.split('>').explode()).reset_index()
#dfs.index.name = 'index'
dfs.head()

,org_id,sample_id,person_id,primary_activity,Work,School,University,Shopping,Leisure,Accompany,Other,Home,iris_chain
0,0,18,1,Work,674822204,NaN,NaN,NaN,NaN,NaN,NaN,671300102,671300102
1,0,18,1,Work,674822204,NaN,NaN,NaN,NaN,NaN,NaN,671300102,674822204
2,0,18,1,Work,674822204,NaN,NaN,NaN,NaN,NaN,NaN,671300102,671300102
3,1,18,2,Work,673360000,NaN,NaN,671300103,671300102,671300102,NaN,671300102,671300102
4,1,18,2,Work,673360000,NaN,NaN,671300103,671300102,671300102,NaN,671300102,671300102


### Save the file to csv and xls

In [180]:
# To csv
dfs.to_csv(r'..\output\_results\loc_split_long.csv')

In [179]:
# Save to xls
writer = pd.ExcelWriter(r'..\output\_results\loc_split_long.xlsx')
dfs.to_excel(writer, '{}'.format('loc'))
writer.save()
print('done')

done


### 3. Pivot table

In [1621]:
loc_split = r'X:\Groups\N41\GROUP-ORGA\MOBILITY_cluster\Activities\Traffic_model\a_matsim_pop_gen\matsim_pop_gen_Strasbourg\output\_results\loa_split_chain_iris_activity.csv'
dfa = pd.read_csv(loc_split, sep = ',')
df['iris_chain_n'] = df['iris_chain_n'].astype(str)
dfa

,index,sample_id,person_id,activity_chain,iris_chain,Start Home != End Home,primary_activity,count_activity,count_iris,diff_count,level_10,activity_actual_chain,iris_chain_n,activity_chain_n
0,0,18,1,Home>Work>Home,671300102>674822204>671300102,False,Work,2,2,0,0,01>11>01,671300102,Home
1,0,18,1,Home>Work>Home,671300102>674822204>671300102,False,Work,2,2,0,1,NaN,674822204,Work
2,1,18,2,Home>Accompany>Work>Accompany>Shopping>Accompa...,671300102>671300102>673360000>671300102>671300...,False,Work,8,8,0,0,01>63>11>62>32>63>01>51>01,671300102,Home
3,1,18,2,Home>Accompany>Work>Accompany>Shopping>Accompa...,671300102>671300102>673360000>671300102>671300...,False,Work,8,8,0,1,NaN,671300102,Accompany
4,1,18,2,Home>Accompany>Work>Accompany>Shopping>Accompa...,671300102>671300102>673360000>671300102>671300...,False,Work,8,8,0,2,NaN,673360000,Work
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
43773,9998,30,5,Home>School>Home>Accompany>School>Accompany>Home,674630000>674630000>674630000>674630000>674630...,False,School,6,6,0,5,NaN,674630000,Accompany
43774,9999,30,6,Home>School>School>School>Home,674630000>674630000>674630000>674630000>674630000,False,School,4,4,0,0,01>21>22>21>01,674630000,Home
43775,9999,30,6,Home>School>School>School>Home,674630000>674630000>674630000>674630000>674630000,False,School,4,4,0,1,NaN,674630000,School
43776,9999,30,6,Home>School>School>School>Home,674630000>674630000>674630000>674630000>674630000,False,School,4,4,0,2,NaN,674630000,School


In [1622]:
#create pivot table
piv_tab = pd.pivot_table(data= dfa, 
                         index = ['iris_chain_n'],
                         values = ['index'],
                         #columns = ['index'],
                         aggfunc=['count'])
print(piv_tab)
# To csv
piv_tab.to_csv(r'..\output\_results\pivot_iris_chain_count.csv')

             count
             index
iris_chain_n      
670010000      163
670020000       10
670030000       53
670040000       25
670060000        3
...            ...
675540000       15
675550000       37
675570000       20
675580000        8
675600000        1

[677 rows x 1 columns]


In [1579]:
#create pivot table
piv_tab2 = pd.pivot_table(data= dfa, 
                          index = ['iris_chain_n','activity_chain_n'], 
                          values =['index'],
                          #columns =['Work','School', 'University','Shopping','Leisure', 'Accompany', 'Other', 'Home'],
                          aggfunc=['count'])
print(piv_tab2)
piv_tab2.to_csv(r'..\output\_results\pivot_iris_chain_activity_count.csv')

                              count
                              index
iris_chain_n activity_chain_n      
670010000    Accompany           14
             Home                82
             Leisure             18
             Other                1
             School              24
...                             ...
675580000    Home                 2
             Leisure              3
             School               1
             Work                 2
675600000    Work                 1

[3289 rows x 1 columns]


In [1584]:
#create pivot table
piv_tab4 = pd.pivot_table(data= dfa, 
                          index = ['activity_chain_n'], 
                          values =['iris_chain_n'],
                          #columns =['iris_chain_n'],
                          aggfunc=['count'])
print(piv_tab4)
piv_tab4.to_csv(r'..\output\_results\pivot_activity_count.csv')

                        count
                 iris_chain_n
activity_chain_n             
Accompany                5726
Home                    17197
Leisure                  6246
Other                     599
School                   3440
Shopping                 4654
University                 81
Work                     5835


In [1612]:
#create pivot table
#dfa= dfa.head(10)
piv_tab5 = pd.pivot_table(data= dfa, 
                          index = ['iris_chain_n'], 
                          values =['index'],
                          columns =['activity_chain_n'],
                          aggfunc=['count'],
                         fill_value=0)#.stack()
#dfa.reset_index(inplace=True)
dfa.columns.name = None
print(piv_tab5)
piv_tab5.to_csv(r'..\output\_results\_pivot_activity_iris.csv')

                     count                                                   
                     index                                                   
activity_chain_n Accompany Home Leisure Other School Shopping University Work
iris_chain_n                                                                 
670010000               14   82      18     1     24       14          0   10
670020000                1    5       1     0      0        0          0    3
670030000                7   25       3     4      9        2          0    3
670040000                0   22       2     0      0        0          0    1
670060000                0    1       2     0      0        0          0    0
...                    ...  ...     ...   ...    ...      ...        ...  ...
675540000                0    8       5     0      0        0          0    2
675550000                4   33       0     0      0        0          0    0
675570000                3   15       2     0      0        0   